In [1]:
import shutil
import re 
from glob import glob
import os
from tqdm.notebook import tqdm

import librosa
import soundfile as sf
import numpy as np 
import random
import pandas as pd

In [2]:
paths = {"sample1": "C:/Users/b.smadiarov/Diploma/DualPathRNN/data/samples/5ed8a1c0f3ea2.flac",
         "sample2": "C:/Users/b.smadiarov/Diploma/DualPathRNN/data/samples/5f2b0a06ceb18.flac",
         "mixed": "/home/berik/workspace/Diploma/DualPathRNN/data/mixed"
        }

glob_paths = {'KS2_raw': 'F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/**/**/*.flac'} 

seed = 42
sr = 16000

In [3]:
random.seed(seed)
np.random.seed(seed)

#### Generate samples

Prepare sourced folder

In [21]:
list_folders = set(['crowdsourced', 'tv_show', 'poition']) 

# Критерий: нету посторонних звуков 
# Выбери какие тебе подойдут для лейблов. Но если не подойдет то можно читать датафрейм и искать все такие видео которые имеют 
#три интервала - "не говорит, говорит, не говорит" - поэтому ты и выбрал crowdsourced.

In [3]:
# Если не жалко памяти.
destination_base = 'F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/Targets/' # так как destination base может поменяться.

# По хорошему лучше их объединить в одну папку - просто дав им разные имена
for t in glob(glob_paths['KS2_raw']):
    t = t.replace('\\', '/') # 'F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/Train/crowdsourced/475mnd37hvnf3t94.flac'
    splited_t = set(t.split('/'))
    if splited_t.intersection(list_folders):
        # shutil.move(t, destination_base + re.sub(r'.*ISSAI_KSC2/', '', t).replace('/', '-'))
        print("Файл успешно перемещен!")
        break

**Generate samples**

In [16]:
def get_file_name(file_path: str):
    return os.path.splitext(os.path.basename(file_path))[0]

In [5]:
def overlap(audio_1: str, audio_2: str, sr=16000):
    y1, sr1 = librosa.load(audio_1, sr=sr)
    y2, sr2 = librosa.load(audio_2, sr=sr)
    assert sr1 == sr2, "Different sample rates!"

    # Idea 1.
    # min_length = min(len(y1), len(y2))
    # y1 = y1[:min_length]
    # y2 = y2[:min_length]
    # # Idea 2.
    # silence_threshold = 1e-5  
    # is_silent_1 = np.all(np.abs(y1[-int(sr*0.1):]) < silence_threshold)
    # is_silent_2 = np.all(np.abs(y2[-int(sr*0.1):]) < silence_threshold)
    # if len(y1) < len(y2):
    #     y1 = np.pad(y1, (0, len(y2) - len(y1))) if is_silent_1 else y1
    # else:
    #     y2 = np.pad(y2, (0, len(y1) - len(y2))) if is_silent_2 else y2

    # Idea 3.
    if len(y1) < len(y2):
        y1 = np.pad(y1, (0, len(y2) - len(y1)))
    else:
        y2 = np.pad(y2, (0, len(y1) - len(y2)))

    overlay = y1 + y2
    return overlay

In [ ]:
# скорее всего его нужно будет перемешать - так как говорящие идут одним за одним.
tracklist = glob('F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/Targets/*.flac')
random.shuffle(tracklist)

In [ ]:
# Короткие записи сливались с короткими записями. 
# Есть ли разница в том что в начале двух аудио тишина например. Возможно стоит обрезать концы и начало аудио. 
# Нужно ли что вообще участков где бы человек говорил - или только speech учатски сигнала были. 

In [12]:
# ЗАМЕЧАНИЕ: ПОРАЖДАЮ Я ИЗ ПАПКИ TARGETS! - ПОЭТОМУ ПУТИ МЕНЯ НЕ НУЖНО. 
# Не забудь добавить tqdm.
# Берем парами 

tracklist = ['/home/berik/workspace/Diploma/DualPathRNN/data/samples/5ed8a1c0f3ea2.flac', 
            '/home/berik/workspace/Diploma/DualPathRNN/data/samples/5f2b0a06ceb18.flac']

# for t in tqdm(tracklist):
for i in range(0, len(tracklist)-1, 2):
    t1, t2 = tracklist[i], tracklist[i+1]
    overlay = overlap(t1, t2)
    audio_name = f"{paths['mixed']}/{get_file_name(t1)}_{get_file_name(t2)}.flac"
    sf.write(f"{audio_name}", overlay, samplerate=16000)
    df = pd.DataFrame({'mixed_audio': [audio_name], 'audio_1': [t1], 'audio_2': [t2]})
    df.to_csv(f"{audio_name.replace('.flac', '.csv')}", index=False)

In [ ]:
# Get full dataset DF
annotations = glob('/home/berik/workspace/Diploma/DualPathRNN/data/mixed/*.csv')
all_dfs = []
for csv in tqdm(annotations):
    df = pd.read_csv(csv)
    all_dfs.append(df)

concated_df = pd.concat(all_dfs).reset_index(drop=True)
concated_df.to_csv('/home/berik/workspace/Diploma/DualPathRNN/data/DATA_INFO/CONCATED_DFS.csv', index=False)

In [ ]:
# Загрузить аудио
# Загрузить его аннотацию
# Получить аннотационные интервалы
# Найти время участок где человек не говорит в начала/конце
# Записать это аудио. 

In [20]:
# Добавить этот код в функцию overlap. 

audio_name = '/home/berik/workspace/Diploma/DualPathRNN/data/samples/5ed8a1c0f3ea2.flac'
audio, sr = librosa.load(audio_name, sr=16000) 
annotation_csv = audio_name.replace('.flac', '.csv') 
df = pd.read_csv(annotation_csv)

# df.head()

# DataFrame имеет следующую структуру: speech | start_time | end_time | utt_time | audio_id
# нужно взять первый speech == '1' и у него "start". Затем нужно взять последний speech == '1' и взять у него "end".
# Делаем именно таким образом - т.к. участок где человек не говорит может не быть, тогда начало и конец должны совпадать.

start = df[df['speech'] == 1].iloc[0]['start_time']
end = df[df['speech'] == 1].iloc[-1]['end_time']

print('start', start, start * sr)
print('end', end, end * sr)
print(10.155 * sr, len(audio)-1)
    
# После этого мы знаем start, end - это временные точки. Теперь нужно обрезать исходное аудио в отрезке [start, end]
trimmed_audio = audio[int(start * sr):int(end * sr)]

new_audio_name = audio_name.replace(f"{get_file_name (audio_name)}", f"trimmed_{get_file_name (audio_name)}") 
# Если нужно сохранить обрезанное аудио, используем следующий код
sf.write(new_audio_name, trimmed_audio, sr)

start 1.7 27200.0
end 9.25 148000.0
162480.0 162474


In [10]:
df.head()

,speech,start_time,end_time,utt_time,audio_id
0,0,0.00,1.700,1.700,F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/Train/crowds...
1,1,1.70,9.250,7.550,F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/Train/crowds...
2,0,9.25,10.155,0.905,F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/Train/crowds...
